In [1]:
import os
import h5py
import uproot
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:

for energy in tqdm(range(1,11,1)):

    pmt_charges = np.empty([0,96,1])
    jade_vtx = np.empty([0,3])
    real_vtx = np.empty([0,3])
    for i in tqdm(range(101)):
        load_path = '/store/hep/users/yewzzang/JSNS2/rat_mc_positron/positron_'+str(energy)+'MeV/jade/positron_'+str(energy)+'MeV_'+str(i)+'.root'
        load_path2 = '/store/hep/users/yewzzang/JSNS2/rat_mc_positron/positron_'+str(energy)+'MeV/jade/reco/Reco_positron_'+str(energy)+'MeV_'+str(i)+'.root'
        load_path3 = '/users/yewzzang/work/make_mc_vertex/analysis/mc_data_1207/event_csv/mcEvent_pmtcharge_jsns2_'+str(energy)+'MeV_'+str(i)+'.csv'


        f = uproot.open(load_path)
        ff = uproot.open(load_path2)
        fff = pd.read_csv(load_path3)



        for events in range(np.array(f['triggerTree/trigger/eventsID/eventsID.pmtCharges']).shape[0]):
            test = ff['recoTriggerTree;1']['recoTrigger/events/events.vertexReco'].array()
            if (np.array(test[events]).shape[0] == 1):



                jade_vtx = np.concatenate((jade_vtx,np.array([np.array(test[events])[0][0],np.array(test[events])[0][1],np.array(test[events])[0][2]]).reshape(1,3)),axis = 0)
                real_vtx = np.concatenate((real_vtx,np.array(fff)[events][-4:-1].reshape(1,3)))
                charges = []
                for pmtid in range(96):




                    charges.append(np.array(f['triggerTree/trigger/eventsID/eventsID.pmtCharges'])[events][0][pmtid])


                pmt_charges = np.concatenate([pmt_charges, np.array(charges).reshape(-1,96,1)])


    pmt_charges = pmt_charges[np.abs(real_vtx[:,2])<1250]
    jade_vtx = jade_vtx[np.abs(real_vtx[:,2])<1250]
    real_vtx = real_vtx[np.abs(real_vtx[:,2])<1250]


    output = '/users/yewzzang/work/make_mc_vertex/analysis/mc_data_1207/positron_'+str(energy)+'MeV.h5'

    kwargs = {'dtype':'f4', 'compression':'lzf'}

    with h5py.File(output, 'w', libver='latest', swmr=True) as fout:

        g = fout.create_group('events')
        g.create_dataset('jade_vertex', data=jade_vtx, chunks=(1,3),**kwargs)
        g.create_dataset('vertex', data=real_vtx, chunks=(1,3),**kwargs)
        g.create_dataset('pmtQ',data=pmt_charges,chunks=(1,96,1),**kwargs)


 52%|█████▏    | 53/101 [01:09<01:03,  1.33s/it]


  5%|▍         | 5/101 [00:06<02:05,  1.31s/it]


 59%|█████▉    | 60/101 [01:18<00:54,  1.33s/it]


 12%|█▏        | 12/101 [00:15<01:56,  1.31s/it]


 65%|██████▌   | 66/101 [01:27<00:46,  1.34s/it]


 18%|█▊        | 18/101 [00:23<01:48,  1.31s/it]


100%|██████████| 10/10 [22:22<00:00, 134.23s/it]
